<a href="https://colab.research.google.com/github/brunompasini/pneumonia-detect/blob/main/pneumonia_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## The data
zip file contains 3 folders, train, test and val

In [3]:
"""zip_where='train.zip'
zip_ref = zipfile.ZipFile(zip_where, 'r')
zip_ref.extractall('/train')"""

zip_where='test.zip'
zip_ref = zipfile.ZipFile(zip_where, 'r')
zip_ref.extractall('/test')

zip_where='val.zip'
zip_ref = zipfile.ZipFile(zip_where, 'r')
zip_ref.extractall('/val')
zip_ref.close()

In [4]:
"""train_normal_dir = os.path.join('/tmp/train/NORMAL')
train_pneumonia_dir = os.path.join('/tmp/train/PNEUMONIA')"""

test_normal_dir = os.path.join('/test/NORMAL')
test_pneumonia_dir = os.path.join('/test/PNEUMONIA')

val_normal_dir = os.path.join('/val/NORMAL')
val_pneumonia_dir = os.path.join('/val/PNEUMONIA')

"""print('Total of training normal images:', len(os.listdir(train_normal_dir)))
print('Total of training pneumonia images:', len(os.listdir(train_pneumonia_dir)))"""
print('Total of test normal images:', len(os.listdir(test_normal_dir)))
print('Total of test pneumonia images:', len(os.listdir(test_pneumonia_dir)))
print('Total of val normal images:', len(os.listdir(val_normal_dir)))
print('Total of val pneumonia images:', len(os.listdir(val_pneumonia_dir)))

FileNotFoundError: ignored

## The model

In [15]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/train/',  # source training images
        target_size=(300, 300),  # resize
        batch_size=128,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        '/test/',
        target_size=(300, 300),
        batch_size=128,
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        '/val/',
        target_size=(300, 300),
        batch_size=32,
        class_mode='binary')

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

In [ ]:
plt.plot(history.history['accuracy'], history.history['val_accuracy'])

In [ ]:
results = model.predict(test)

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  path = '/running/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is normal")
  else:
    print(fn + " is pneumonia")